# FDM Example
เอกสารนี้เตรียมไว้เป็นตัวอย่างสำหรับการทดลองถอดรหัสข้อมูลที่ถูกมัลติเพล็กซ์ไว้ด้วยกลไก Frequency Division Multiplexing (FDM)

In [ ]:
from numpy import sin,cos,pi
from sigproc import Signal,start_notebook
bkp = start_notebook()

## อ่านอินพุทไฟล์
* อินพุทไฟล์เป็นสัญญาณที่เกิดจากการผสมข้อมูลจาก 3 แหล่งด้วยกระบวนการมอดูเลชันแบบ QPSK ที่ความถี่ต่าง ๆ กัน
* เมื่ออ่านอินพุทไฟล์และลองแสดงค่าสัญญาณในแกนเวลา จะเห็นได้ว่าไม่สามารถตีความข้อมูลภายในสัญญาณด้วยตาเปล่าได้เนื่องจากมีการผสมกันของข้อมูลหลายชุดที่กระจายกันอยู่ที่ความถี่ต่าง ๆ
* เมื่อตรวจสอบสัญญาณในโดเมนความถี่ จะเห็นว่ามีสัญญาณกระจายตัวอยู่ 3 ช่องสัญญาณย่อย ที่ความถี่พาหะ 100 Hz, 200 Hz และ 300 Hz แต่ละช่องสัญญาณย่อยมีแบนด์วิธด์ประมาณ 80 Hz

In [ ]:
s = Signal(wav_file="example.wav")
s.plot_time(fig_options=dict(height=200))
s.plot_freq(fig_options=dict(x_range=(0,500),height=200))

## คัดกรองช่องสัญญาณที่ต้องการ
* โค้ดด้านล่างคัดลอกสัญญาณขึ้นมาใหม่และกรองออกมาเฉพาะสัญญาณในย่าน 160 - 240 Hz โดยใช้เมท็อด `filter()` ที่มีอยู่ในคลาส `Signal`
* ค่าสัญญาณในโดเมนความถี่แสดงให้เห็นว่าสัญญาณเหลือเพียงความถี่เฉพาะในย่าน 160 - 240 Hz เท่านั้น
* ค่าสัญญาณในโดเมนเวลาเริ่มแยกความแตกต่างของซิมโบลได้ด้วยตาเปล่าได้บ้าง แต่จะเห็นว่าสัญญาณบิดเบี้ยวไปบางส่วนเนื่องจากสัญญาณดั้งเดิมผ่านการกรองด้วย band-pass filter ก่อนเข้ากระบวนการ FDM เพื่อไม่ให้กวนช่องสัญญาณใกล้เคียง อย่างไรก็ตามเราจะถอดสัญญาณ baseband ออกมาจากสัญญาณนี้ก่อนเพื่อให้การตีความง่ายขึ้น

In [ ]:
ch2 = s.copy().filter((160,240))
ch2.plot_freq(fig_options=dict(x_range=(0,500),height=200))
ch2.plot_time(fig_options=dict(height=200))

## ถอดสัญญาณ baseband
* สัญญาณ baseband ทั้งส่วนของ I และ Q ทำได้โดยการคูณสัญญาณเข้ากับคลื่นพาหะที่มุม 0 และ 90 องศา
* ผลลัพธ์จากการคูณสัญญาณต้องนำมาผ่าน low-pass filter เพื่อให้ได้สัญญาณ baseband ออกมา

In [ ]:
carrier_i = Signal(duration=ch2.duration,func=lambda t:sin(2*pi*200*t))
carrier_q = Signal(duration=ch2.duration,func=lambda t:cos(2*pi*200*t))
baseband_i = (ch2*carrier_i*2).filter(200)
baseband_q = (ch2*carrier_q*2).filter(200)
print("Baseband I")
baseband_i.plot_time(fig_options=dict(height=200))
print("Baseband Q")
baseband_q.plot_time(fig_options=dict(height=200))